In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = "1"
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow_model_optimization.python.core.quantization.keras.quantizers import Quantizer
from tensorflow_model_optimization.python.core.quantization.keras.quantize_config import QuantizeConfig
from tensorflow_model_optimization.python.core.quantization.keras.quantizers import LastValueQuantizer, MovingAverageQuantizer
from tensorflow_model_optimization.python.core.quantization.keras.quantize import quantize_annotate_layer, quantize_apply
from tensorflow_model_optimization.quantization.keras import quantize_apply, quantize_scope
import tensorflow_model_optimization as tfmot
from tensorflow.python.profiler import model_analyzer
from tensorflow.python.profiler import option_builder


2025-02-18 14:09:32.760397: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-18 14:09:32.769075: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739884172.778592    8236 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739884172.781613    8236 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-18 14:09:32.792543: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
#Generate test data:
X=np.random.rand(10000,5)
y = np.sum(X, axis=1)
print(np.shape(X))
print(np.shape(y))

(10000, 5)
(10000,)


In [3]:
class FixedRangeQuantizer(Quantizer):
    def build(self, tensor_shape, name, layer):
        range_var = layer.add_weight(
            name=name + '_range',
            initializer=tf.keras.initializers.Constant(6.0),
            trainable=False
        )
        return {'range_var': range_var}

    def __call__(self, inputs, training, weights, **kwargs):
        return tf.keras.backend.clip(inputs, 0.0, weights['range_var'])

    def get_config(self):
        return {}

In [4]:


class CustomLayerQuantizeConfig(QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return [
        (layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False)),
        (layer.bias,   LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False)),
    ]

    def get_activations_and_quantizers(self, layer):
        #return []
        return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
        layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
        layer.activation = quantize_activations[0]

    def get_output_quantizers(self, layer):
        return [MovingAverageQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False)]
        #return []

    def get_config(self):
        return {}


In [5]:
model2 = tf.keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=(5,)),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='relu')
])
#quantize_model = tfmot.quantization.keras.quantize_model
#model2 = quantize_model(model2)

2025-02-18 14:09:34.339513: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1739884174.340408    8236 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9


In [6]:
model = tf.keras.Sequential([
    quantize_annotate_layer(layers.Dense(1024, activation='relu', input_shape=(5,)), quantize_config=CustomLayerQuantizeConfig()),
    quantize_annotate_layer(layers.Dense(512, activation='relu'), quantize_config=CustomLayerQuantizeConfig()),
    quantize_annotate_layer(layers.Dense(256, activation='relu'), quantize_config=CustomLayerQuantizeConfig()),
    quantize_annotate_layer(layers.Dense(128, activation='relu'), quantize_config=CustomLayerQuantizeConfig()),
    quantize_annotate_layer(layers.Dense(64, activation='relu'), quantize_config=CustomLayerQuantizeConfig()),
    quantize_annotate_layer(layers.Dense(1, activation='relu'), quantize_config=CustomLayerQuantizeConfig()),
    #layers.Dense(100, activation='relu', input_shape=(5,)),
    #layers.Dense(1, activation='relu')
])
#with quantize_scope({'CustomLayerQuantizeConfig': CustomLayerQuantizeConfig}):
#    quant_aware_model = quantize_apply(model)

model = tf.keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=(5,)),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='relu')
])
quantize_model = tfmot.quantization.keras.quantize_model
quant_aware_model = quantize_model(model)

In [7]:
import quantize_model
epochs=10
batch_size=64
num_training_samples=len(X)
print(int(num_training_samples/batch_size*epochs))
model2, callbacks=quantize_model.get_pruning_wrapper(model2, 0.5, epochs,batch_size,num_training_samples)
model2.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
print(np.shape(X))

model2.fit(X, y,callbacks=callbacks, epochs=epochs, batch_size=batch_size)
quant_aware_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
quant_aware_model.fit(X, y, epochs=epochs)

1562
(10000, 5)
Epoch 1/10


I0000 00:00:1739884177.109546    8348 service.cc:148] XLA service 0x7d9d853ebef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739884177.109564    8348 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Ti, Compute Capability 8.9
2025-02-18 14:09:37.112888: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1739884177.122428    8348 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1739884177.168621    8348 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


157/157 [==============================] - 3s 2ms/step - loss: 0.2073 - accuracy: 0.0000e+00
Epoch 2/10
157/157 [==============================] - 0s 2ms/step - loss: 5.4538e-05 - accuracy: 0.0000e+00
Epoch 3/10
157/157 [==============================] - 0s 2ms/step - loss: 2.7801e-05 - accuracy: 0.0000e+00
Epoch 4/10
157/157 [==============================] - 0s 2ms/step - loss: 1.7523e-05 - accuracy: 0.0000e+00
Epoch 5/10
157/157 [==============================] - 1s 7ms/step - loss: 1.4200e-05 - accuracy: 0.0000e+00
Epoch 6/10
157/157 [==============================] - 0s 2ms/step - loss: 6.7275e-04 - accuracy: 0.0000e+00
Epoch 7/10
157/157 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 0.0000e+00
Epoch 8/10
157/157 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 0.0000e+00
Epoch 9/10
157/157 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 0.0000e+00
Epoch 10/10
157/157 [==============================] - 

In [8]:
model2 = quantize_model.strip_prune(model2)

In [9]:
vals=np.random.rand(1,5)
result=np.sum(vals)
prediction=quant_aware_model.predict(vals)
prediction2=model2.predict(vals)
print(result)
print(prediction)
print(prediction2)

1/1 [==============================] - 0s 30ms/step
3.1012395830769353
[[3.1076546]]
[[3.1029394]]


In [10]:
index = np.random.choice(X.shape[0], 100, replace=False)
x_random = X[index]
def representative_data_gen():
    # Here, let's use 100 samples for calibration
    for i in range(100):
        # The model expects (batch_size=1, 5) if it’s Dense(…, input_shape=(5,)).
        # So we add a batch dimension of size 1:
        yield [x_random[i:i+1].astype(np.float32)]  # shape (1, 5)

In [11]:
quantconverter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)

quantize_model.quantize_8_bit(model2,x_random, "testfolder/32bit")


quantconverter.optimizations = [tf.lite.Optimize.DEFAULT]
quantconverter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
quantconverter.representative_dataset = representative_data_gen
quantlite=quantconverter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp8gux02z_/assets


INFO:tensorflow:Assets written to: /tmp/tmp8gux02z_/assets
/home/henrik/anaconda3/envs/tf-gpu/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1739884188.701715    8236 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1739884188.701724    8236 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-02-18 14:09:48.701876: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp8gux02z_
2025-02-18 14:09:48.702294: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-02-18 14:09:48.702301: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp8gux02z_
I0000 00:00:1739884188.705277    8236 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-02-18 14:09:48.705634: I tensorflow/cc/saved

INFO:tensorflow:Assets written to: /tmp/tmpot3zqgwz/assets


INFO:tensorflow:Assets written to: /tmp/tmpot3zqgwz/assets
/home/henrik/anaconda3/envs/tf-gpu/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1739884189.723732    8236 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1739884189.723742    8236 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-02-18 14:09:49.723837: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpot3zqgwz
2025-02-18 14:09:49.725528: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-02-18 14:09:49.725536: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpot3zqgwz
2025-02-18 14:09:49.736766: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-02-18 14:09:49.786227: I tensorflow/cc/saved_model/loader.cc

In [12]:
#quant_aware_model.save('testfolder/8bit.h5')  # Save the model in HDF5 format
#model2.save('testfolder/32bit.h5')  # Save the model in HDF5 format
with open("testfolder/8bit.tflite", "wb") as f:
    f.write(quantlite)

# Get the size of the saved model file in bytes
model_size = os.path.getsize('testfolder/8bit.tflite')
print(model_size)

model_size = os.path.getsize('testfolder/32bit.tflite')
print(model_size)

714320
760976


Before:
714344
761032

In [13]:
import lzma

with open("testfolder/8bit.tflite", "rb") as f_in:
    model_data = f_in.read()

compressed_data = lzma.compress(model_data)

with open("testfolder/8bit.tflite.xz", "wb") as f_out:
    f_out.write(compressed_data)


In [14]:
with open("testfolder/32bit.tflite", "rb") as f_in:
    model_data = f_in.read()

compressed_data = lzma.compress(model_data)

with open("testfolder/32bit.tflite.xz", "wb") as f_out:
    f_out.write(compressed_data)